In [1]:
import pandas as pd
import importlib
import os
from ast import literal_eval
import numpy as np
import pickle
import gc
from sklearn.model_selection import train_test_split


In [3]:
#df.to_pickle('../../data/song_train_merged_transferkey_pkl0430') 
df = pd.read_pickle('../../../data/song_train_merged_transferkey_pkl0602')

In [33]:
df['rating']=df['rating'].str.replace(',','')

# df cleaning

In [34]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,artist,chord,error,rating,tab_href,tab_title,version,song_name,song_detector_chord,song_chord,chord_before_tf
0,0,0,28,5cada0049fc5af34a460ae05,10cc,"[G:maj, C:maj, G:maj, Eb:maj, G:maj, G:maj, N,...",0,9,https://tabs.ultimate-guitar.com/tab/10cc/art_...,Art For Arts Sake,Ver 1,Art For Arts Sake 10cc,"[{'st': 0, 'et': 0.511, 'ochord': 'N'}, {'st':...","[N, D:maj, F:maj, D:maj, G:maj, G:maj, Db:maj,...","[G:maj, C:maj, G:maj, Eb:maj, G:maj, G:maj, F:..."
1,1,1,29,5cada0049fc5af34a460ae07,10cc,"[G:maj, N, C:maj, Bb:maj, N, G:maj, C:maj, A:m...",0,3,https://tabs.ultimate-guitar.com/tab/10cc/baro...,Baron Samedi,Ver 1,Baron Samedi 10cc,"[{'st': 0, 'et': 3.413, 'ochord': 'A:min'}, {'...","[A:min, G:maj, F:maj, C:maj, D:maj, C:maj, Bb:...","[A:min, F:maj, D:maj, Bb:maj, N, A:min, D:maj,..."
2,2,2,30,5cada0049fc5af34a460ae0b,10cc,"[Bb:maj, Bb:maj, C:maj, C:maj, Bb:maj, Bb:maj,...",0,4,https://tabs.ultimate-guitar.com/tab/10cc/bran...,Brand New Day,Ver 1,Brand New Day 10cc,"[{'st': 0, 'et': 0.58, 'ochord': 'C'}, {'st': ...","[C:maj, F:min, Bb:maj, G:maj, C:maj, F:min, Bb...","[Bb:maj, Bb:maj, C:maj, C:maj, Bb:maj, Bb:maj,..."
3,3,3,31,5cada0049fc5af34a460ae0d,10cc,"[Bb:maj, Eb:maj, Bb:maj, C:maj, G:maj, C:maj, ...",0,5,https://tabs.ultimate-guitar.com/tab/10cc/donn...,Donna,Ver 1,Donna 10cc,"[{'st': 0, 'et': 5.782, 'ochord': 'A#'}, {'st'...","[Bb:maj, Bb:maj, C:maj, Eb:maj, Bb:maj, C:maj,...","[A:maj, D:maj, A:maj, B:maj, Gb:min, B:maj, Gb..."
4,4,4,32,5cada0049fc5af34a460ae0f,10cc,"[C:maj, A:min, G:maj, F:maj, A:min, D:maj, A:m...",0,75,https://tabs.ultimate-guitar.com/tab/10cc/drea...,Dreadlock Holiday,Ver 1,Dreadlock Holiday 10cc,"[{'st': 0, 'et': 1.254, 'ochord': 'N'}, {'st':...","[N, B:maj, Bb:maj, G:maj, C:maj, G:maj, C:maj,...","[G:maj, E:min, D:maj, C:maj, E:min, A:min, E:m..."


把rating欄位轉成float型態，轉完後可能會有一些row變成NAN，把這些row變成0.0

In [35]:

df['rating']= pd.to_numeric(df['rating'], errors='coerce').fillna(0)

清除outlier:任何一首歌曲和弦數目太多或太少都不要，以便之後做padding <br>
用四分位數檢視

In [9]:
def status(x) : 
    return pd.Series([x.count(),x.min(),x.idxmin(),x.quantile(.05),x.quantile(.25),x.median(),
                      x.quantile(.75),x.quantile(.95),x.mean(),x.max(),x.idxmax(),x.mad(),x.var(),
                      x.std(),x.skew(),x.kurt()],index=['總數','最小值','最小值位置','5%分位數','25%分位數',
                    '中位數','75%分位數','95%分位數','均值','最大值','最大值位數','平均絕對偏差','方差','標準差','偏度','峰度'])


In [37]:
print(status(df['song_chord'].str.len()),status(df['chord'].str.len()))


總數        20102.000000
最小值          10.000000
最小值位置      9269.000000
5%分位數        49.000000
25%分位數       75.000000
中位數          95.000000
75%分位數      117.000000
95%分位數      157.000000
均值          104.911452
最大值        3954.000000
最大值位數     16425.000000
平均絕對偏差       33.794768
方差        12884.677796
標準差         113.510695
偏度           15.669242
峰度          303.136858
dtype: float64 總數        20102.000000
最小值           0.000000
最小值位置        87.000000
5%分位數        24.000000
25%分位數       55.000000
中位數          80.000000
75%分位數      106.000000
95%分位數      159.000000
均值           83.809919
最大值         770.000000
最大值位數      1938.000000
平均絕對偏差       32.350549
方差         1910.253356
標準差          43.706445
偏度            1.618664
峰度            9.751067
dtype: float64


發現chord欄位的95%是159，chord欄位的5%是24 <br>
選擇此兩區間以去除outlier

In [38]:
index=(df['song_chord'].str.len()<159)  & (df['song_chord'].str.len()>24) 
df=df[index]
index=(df['chord'].str.len()<159)  & (df['chord'].str.len()>24) 
df=df[index]

In [39]:
print(status(df['song_chord'].str.len()),status(df['chord'].str.len()))


總數        17309.000000
最小值          25.000000
最小值位置       797.000000
5%分位數        50.000000
25%分位數       75.000000
中位數          93.000000
75%分位數      112.000000
95%分位數      140.000000
均值           93.656133
最大值         158.000000
最大值位數      1541.000000
平均絕對偏差       21.821889
方差          716.681379
標準差          26.770905
偏度            0.061189
峰度           -0.479872
dtype: float64 總數        17309.000000
最小值          25.000000
最小值位置       685.000000
5%分位數        34.000000
25%分位數       58.000000
中位數          79.000000
75%分位數      103.000000
95%分位數      136.000000
均值           81.226703
最大值         158.000000
最大值位數      1650.000000
平均絕對偏差       25.164095
方差          934.335014
標準差          30.566894
偏度            0.282784
峰度           -0.611973
dtype: float64


In [40]:
print(status(df['chord_before_tf'].str.len()))

總數        17309.000000
最小值          25.000000
最小值位置       685.000000
5%分位數        34.000000
25%分位數       58.000000
中位數          79.000000
75%分位數      103.000000
95%分位數      136.000000
均值           81.226703
最大值         158.000000
最大值位數      1650.000000
平均絕對偏差       25.164095
方差          934.335014
標準差          30.566894
偏度            0.282784
峰度           -0.611973
dtype: float64


# 以上區塊做完df cleaning，存起來之後就直接讀取

In [3]:
#df.to_pickle('../../data/model_df_cleaned') 
df = pd.read_pickle('../../../../data/model_df_cleaned0609')

In [4]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,artist,chord,error,rating,tab_href,tab_title,...,song_name,song_detector_chord,song_chord,chord_before_tf,song_chord_toCkey,star,visitors_0530,tab_dtw_cost,chord_alignment_by_dtw,songchord_alignment_by_dtw
0,69,69,103,5cada0119fc5af34a460af19,a-ha,"[E:maj, D:maj, A:maj, D:maj, E:maj, D:maj, A:m...",0,6,https://tabs.ultimate-guitar.com/tab/a-ha/trai...,Train Of Thought,...,Train Of Thought a-ha,"[{'st': 0, 'et': 3.762, 'ochord': 'N'}, {'st':...","[N, N, N, C:maj, D:maj, N, E:min, D:maj, E:maj...","[E:maj, D:maj, A:maj, D:maj, E:maj, D:maj, A:m...","[N, N, N, N, F:maj, N, G:min, F:maj, G:maj, N,...",4.5,6,49.8,"[E:maj, N, N, N, D:maj, N, A:maj, D:maj, E:maj...","[N, N, N, C:maj, D:maj, N, E:min, D:maj, E:maj..."
1,70,70,104,5cada0119fc5af34a460af1d,a-ha,"[Gb:min, A:maj, A:maj, D:maj, G:maj, Gb:min, A...",0,4,https://tabs.ultimate-guitar.com/tab/a-ha/unde...,Under The Makeup,...,Under The Makeup a-ha,"[{'st': 0, 'et': 7.384, 'ochord': 'N'}, {'st':...","[N, N, Gb:maj, A:maj, D:maj, G:maj, B:maj, Gb:...","[Gb:min, A:maj, A:maj, D:maj, G:maj, Gb:min, A...","[N, N, E:maj, G:maj, C:maj, F:maj, A:maj, E:ma...",4.0,5,39.3,"[Gb:min, N, N, A:maj, A:maj, D:maj, G:maj, N, ...","[N, N, Gb:maj, A:maj, N, D:maj, G:maj, B:maj, ..."
2,71,71,105,5cada0119fc5af34a460af1f,a-ha,"[Db:maj, Db:min, Db:maj, Gb:min, A:maj, D:maj,...",0,7,https://tabs.ultimate-guitar.com/tab/a-ha/unde...,Under The Makeup,...,Under The Makeup a-ha,"[{'st': 0, 'et': 7.384, 'ochord': 'N'}, {'st':...","[N, N, Gb:maj, A:maj, D:maj, G:maj, B:maj, Gb:...","[Db:maj, Db:min, Db:maj, Gb:min, A:maj, D:maj,...","[N, N, E:maj, G:maj, C:maj, F:maj, A:maj, E:ma...",4.5,7,38.5,"[Db:maj, Db:min, Db:maj, Gb:min, A:maj, D:maj,...","[N, N, Gb:maj, N, A:maj, D:maj, G:maj, B:maj, ..."
3,72,72,106,5cada0129fc5af34a460af23,a-ha,"[G:maj, D:maj, A:maj, C:maj, G:maj, D:maj, A:m...",0,29,https://tabs.ultimate-guitar.com/tab/a-ha/velv...,Velvet,...,Velvet a-ha,"[{'st': 0, 'et': 4.365, 'ochord': 'C'}, {'st':...","[C:maj, C:maj, C:maj, A:min, C:maj, F:maj, C:m...","[C:maj, G:maj, D:maj, F:maj, C:maj, G:maj, D:m...","[F:maj, F:maj, F:maj, D:min, F:maj, N, F:maj, ...",4.5,29,45.0,"[G:maj, N, D:maj, A:maj, C:maj, N, N, G:maj, D...","[C:maj, C:maj, C:maj, A:min, C:maj, F:maj, C:m..."
4,73,73,107,5cada0129fc5af34a460af25,a-ha,"[G:maj, D:maj, A:min, C:maj, G:maj, D:maj, A:m...",0,11,https://tabs.ultimate-guitar.com/tab/a-ha/velv...,Velvet,...,Velvet a-ha,"[{'st': 0, 'et': 4.365, 'ochord': 'C'}, {'st':...","[C:maj, C:maj, C:maj, A:min, C:maj, F:maj, C:m...","[G:maj, D:maj, A:min, C:maj, G:maj, D:maj, A:m...","[F:maj, F:maj, F:maj, D:min, F:maj, N, F:maj, ...",4.0,11,43.2,"[G:maj, D:maj, A:min, N, C:maj, N, N, G:maj, D...","[C:maj, C:maj, C:maj, A:min, C:maj, F:maj, C:m..."
5,74,74,108,5cada0129fc5af34a460af2d,a-ha,"[Bb:min, Eb:min, Ab:maj, Bb:min, Bb:min, Eb:mi...",0,7,https://tabs.ultimate-guitar.com/tab/a-ha/youl...,Youll Never Get Over Me,...,Youll Never Get Over Me a-ha,"[{'st': 0, 'et': 0.395, 'ochord': 'F#:min'}, {...","[Gb:maj, Ab:maj, Bb:maj, Db:maj, Gb:maj, Ab:ma...","[E:min, A:min, D:maj, E:min, E:min, A:min, G:m...","[F:maj, G:maj, A:maj, C:maj, F:maj, G:maj, G:m...",4.5,7,78.2,"[Bb:min, N, N, N, Eb:min, Ab:maj, N, N, N, N, ...","[Gb:maj, Ab:maj, Bb:maj, Db:maj, Gb:maj, Ab:ma..."
6,75,75,109,5cada0129fc5af34a460af2f,ABBA,"[C:maj, G:maj, F:maj, C:maj, F:maj, C:maj, G:m...",0,91,https://tabs.ultimate-guitar.com/tab/abba/anda...,Andante Andante,...,Andante Andante ABBA,"[{'st': 0, 'et': 1.974, 'ochord': 'D'}, {'st':...","[D:maj, F:maj, C:maj, F:maj, Bb:maj, F:maj, C:...","[F:maj, C:maj, Bb:maj, F:maj, Bb:maj, F:maj, C...","[A:maj, C:maj, G:maj, C:maj, F:maj, C:maj, G:m...",5.0,108,42.6,"[C:maj, G:maj, F:maj, C:maj, F:maj, N, N, C:ma...","[D:maj, N, F:maj, C:maj, F:maj, Bb:maj, F:maj,..."
7,76,76,110,5cada0129fc5af34a460af31,ABBA,"[Gb:maj, E:maj, Gb:maj, E:maj, Gb:maj, B:maj, ...",0,29,https://tabs.ultimate-guitar.com/tab/abba/ange...,Angeleyes,...,Angeleyes 

In [5]:
#17309
len(df)

17240

In [10]:
print('共:',len(df.groupby('song_name').size()),'首歌')
print('其中有:',sum(df.groupby('song_name').size()>1),'首歌擁有兩張以上的譜')

# 0609以前資料
# 共: 9970 首歌
# 其中有: 3517 首歌擁有兩張以上的譜


共: 9933 首歌
其中有: 3510 首歌擁有兩張以上的譜


找出擁有兩張譜以上的歌 <br>
挑前1000首歌出來之後作為模型的test song

In [11]:
#擁有兩張譜以上的歌
#3517筆
has_2ver=df.groupby('song_name').size().loc[lambda x: x>1].keys()
print('共:',len(has_2ver),'首歌擁有兩張以上的譜')


共: 3510 首歌擁有兩張以上的譜


0511:
training song:過濾掉前1000首擁有兩張譜以上的歌(3517的前1000) <br>
test song:選擇前1000首擁有兩張譜以上的歌(3517的前1000)

05/16:
training song:過濾所有擁有兩張譜以上的歌(3517首) <br>
test song:選擇前1000首擁有兩張譜以上的歌(3517的前1000)

In [12]:
df_remove_test=df[~df.song_name.isin(has_2ver[:1000])]
#df_remove_test=df[~df.song_name.isin(has_2ver)]
df_remove_test

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,artist,chord,error,rating,tab_href,tab_title,...,song_name,song_detector_chord,song_chord,chord_before_tf,song_chord_toCkey,star,visitors_0530,tab_dtw_cost,chord_alignment_by_dtw,songchord_alignment_by_dtw
0,69,69,103,5cada0119fc5af34a460af19,a-ha,"[E:maj, D:maj, A:maj, D:maj, E:maj, D:maj, A:m...",0,6,https://tabs.ultimate-guitar.com/tab/a-ha/trai...,Train Of Thought,...,Train Of Thought a-ha,"[{'st': 0, 'et': 3.762, 'ochord': 'N'}, {'st':...","[N, N, N, C:maj, D:maj, N, E:min, D:maj, E:maj...","[E:maj, D:maj, A:maj, D:maj, E:maj, D:maj, A:m...","[N, N, N, N, F:maj, N, G:min, F:maj, G:maj, N,...",4.5,6,49.8,"[E:maj, N, N, N, D:maj, N, A:maj, D:maj, E:maj...","[N, N, N, C:maj, D:maj, N, E:min, D:maj, E:maj..."
1,70,70,104,5cada0119fc5af34a460af1d,a-ha,"[Gb:min, A:maj, A:maj, D:maj, G:maj, Gb:min, A...",0,4,https://tabs.ultimate-guitar.com/tab/a-ha/unde...,Under The Makeup,...,Under The Makeup a-ha,"[{'st': 0, 'et': 7.384, 'ochord': 'N'}, {'st':...","[N, N, Gb:maj, A:maj, D:maj, G:maj, B:maj, Gb:...","[Gb:min, A:maj, A:maj, D:maj, G:maj, Gb:min, A...","[N, N, E:maj, G:maj, C:maj, F:maj, A:maj, E:ma...",4.0,5,39.3,"[Gb:min, N, N, A:maj, A:maj, D:maj, G:maj, N, ...","[N, N, Gb:maj, A:maj, N, D:maj, G:maj, B:maj, ..."
2,71,71,105,5cada0119fc5af34a460af1f,a-ha,"[Db:maj, Db:min, Db:maj, Gb:min, A:maj, D:maj,...",0,7,https://tabs.ultimate-guitar.com/tab/a-ha/unde...,Under The Makeup,...,Under The Makeup a-ha,"[{'st': 0, 'et': 7.384, 'ochord': 'N'}, {'st':...","[N, N, Gb:maj, A:maj, D:maj, G:maj, B:maj, Gb:...","[Db:maj, Db:min, Db:maj, Gb:min, A:maj, D:maj,...","[N, N, E:maj, G:maj, C:maj, F:maj, A:maj, E:ma...",4.5,7,38.5,"[Db:maj, Db:min, Db:maj, Gb:min, A:maj, D:maj,...","[N, N, Gb:maj, N, A:maj, D:maj, G:maj, B:maj, ..."
3,72,72,106,5cada0129fc5af34a460af23,a-ha,"[G:maj, D:maj, A:maj, C:maj, G:maj, D:maj, A:m...",0,29,https://tabs.ultimate-guitar.com/tab/a-ha/velv...,Velvet,...,Velvet a-ha,"[{'st': 0, 'et': 4.365, 'ochord': 'C'}, {'st':...","[C:maj, C:maj, C:maj, A:min, C:maj, F:maj, C:m...","[C:maj, G:maj, D:maj, F:maj, C:maj, G:maj, D:m...","[F:maj, F:maj, F:maj, D:min, F:maj, N, F:maj, ...",4.5,29,45.0,"[G:maj, N, D:maj, A:maj, C:maj, N, N, G:maj, D...","[C:maj, C:maj, C:maj, A:min, C:maj, F:maj, C:m..."
4,73,73,107,5cada0129fc5af34a460af25,a-ha,"[G:maj, D:maj, A:min, C:maj, G:maj, D:maj, A:m...",0,11,https://tabs.ultimate-guitar.com/tab/a-ha/velv...,Velvet,...,Velvet a-ha,"[{'st': 0, 'et': 4.365, 'ochord': 'C'}, {'st':...","[C:maj, C:maj, C:maj, A:min, C:maj, F:maj, C:m...","[G:maj, D:maj, A:min, C:maj, G:maj, D:maj, A:m...","[F:maj, F:maj, F:maj, D:min, F:maj, N, F:maj, ...",4.0,11,43.2,"[G:maj, D:maj, A:min, N, C:maj, N, N, G:maj, D...","[C:maj, C:maj, C:maj, A:min, C:maj, F:maj, C:m..."
5,74,74,108,5cada0129fc5af34a460af2d,a-ha,"[Bb:min, Eb:min, Ab:maj, Bb:min, Bb:min, Eb:mi...",0,7,https://tabs.ultimate-guitar.com/tab/a-ha/youl...,Youll Never Get Over Me,...,Youll Never Get Over Me a-ha,"[{'st': 0, 'et': 0.395, 'ochord': 'F#:min'}, {...","[Gb:maj, Ab:maj, Bb:maj, Db:maj, Gb:maj, Ab:ma...","[E:min, A:min, D:maj, E:min, E:min, A:min, G:m...","[F:maj, G:maj, A:maj, C:maj, F:maj, G:maj, G:m...",4.5,7,78.2,"[Bb:min, N, N, N, Eb:min, Ab:maj, N, N, N, N, ...","[Gb:maj, Ab:maj, Bb:maj, Db:maj, Gb:maj, Ab:ma..."
6,75,75,109,5cada0129fc5af34a460af2f,ABBA,"[C:maj, G:maj, F:maj, C:maj, F:maj, C:maj, G:m...",0,91,https://tabs.ultimate-guitar.com/tab/abba/anda...,Andante Andante,...,Andante Andante ABBA,"[{'st': 0, 'et': 1.974, 'ochord': 'D'}, {'st':...","[D:maj, F:maj, C:maj, F:maj, Bb:maj, F:maj, C:...","[F:maj, C:maj, Bb:maj, F:maj, Bb:maj, F:maj, C...","[A:maj, C:maj, G:maj, C:maj, F:maj, C:maj, G:m...",5.0,108,42.6,"[C:maj, G:maj, F:maj, C:maj, F:maj, N, N, C:ma...","[D:maj, N, F:maj, C:maj, F:maj, Bb:maj, F:maj,..."
9,78,78,112,5cada0129fc5af34a460af37,ABBA,"[Gb:maj, B:maj, Gb:maj, B:maj, Gb:maj, B:maj, ...",0,3,https://tabs.ultimate-guitar.com/tab/abba/as_g...,As Good As New,...,As Goo

In [1]:
print('剩下:',len(df_remove_test.groupby('song_name').size()),'首歌')

NameError: name 'df_remove_test' is not defined

# QA架構:

每組song_name取song_chord作為Q;guitar_tab_chord of Max rating作為good answer  <br>
Training_data:11360首只有一張譜的歌+3145首有兩張以上譜的歌 <br>
Testing_data:1000首擁有兩張以上譜的歌

準備資料:
從df2整理出Q與GoodA


In [14]:
#df2:Good Answer的df，每首歌曲最高rating的譜
idx = df_remove_test.groupby(['song_name'])['rating'].transform(max) == df_remove_test['rating']
df2=df_remove_test[idx].copy()
df2

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,artist,chord,error,rating,tab_href,tab_title,...,song_name,song_detector_chord,song_chord,chord_before_tf,song_chord_toCkey,star,visitors_0530,tab_dtw_cost,chord_alignment_by_dtw,songchord_alignment_by_dtw
0,69,69,103,5cada0119fc5af34a460af19,a-ha,"[E:maj, D:maj, A:maj, D:maj, E:maj, D:maj, A:m...",0,6,https://tabs.ultimate-guitar.com/tab/a-ha/trai...,Train Of Thought,...,Train Of Thought a-ha,"[{'st': 0, 'et': 3.762, 'ochord': 'N'}, {'st':...","[N, N, N, C:maj, D:maj, N, E:min, D:maj, E:maj...","[E:maj, D:maj, A:maj, D:maj, E:maj, D:maj, A:m...","[N, N, N, N, F:maj, N, G:min, F:maj, G:maj, N,...",4.5,6,49.8,"[E:maj, N, N, N, D:maj, N, A:maj, D:maj, E:maj...","[N, N, N, C:maj, D:maj, N, E:min, D:maj, E:maj..."
2,71,71,105,5cada0119fc5af34a460af1f,a-ha,"[Db:maj, Db:min, Db:maj, Gb:min, A:maj, D:maj,...",0,7,https://tabs.ultimate-guitar.com/tab/a-ha/unde...,Under The Makeup,...,Under The Makeup a-ha,"[{'st': 0, 'et': 7.384, 'ochord': 'N'}, {'st':...","[N, N, Gb:maj, A:maj, D:maj, G:maj, B:maj, Gb:...","[Db:maj, Db:min, Db:maj, Gb:min, A:maj, D:maj,...","[N, N, E:maj, G:maj, C:maj, F:maj, A:maj, E:ma...",4.5,7,38.5,"[Db:maj, Db:min, Db:maj, Gb:min, A:maj, D:maj,...","[N, N, Gb:maj, N, A:maj, D:maj, G:maj, B:maj, ..."
3,72,72,106,5cada0129fc5af34a460af23,a-ha,"[G:maj, D:maj, A:maj, C:maj, G:maj, D:maj, A:m...",0,29,https://tabs.ultimate-guitar.com/tab/a-ha/velv...,Velvet,...,Velvet a-ha,"[{'st': 0, 'et': 4.365, 'ochord': 'C'}, {'st':...","[C:maj, C:maj, C:maj, A:min, C:maj, F:maj, C:m...","[C:maj, G:maj, D:maj, F:maj, C:maj, G:maj, D:m...","[F:maj, F:maj, F:maj, D:min, F:maj, N, F:maj, ...",4.5,29,45.0,"[G:maj, N, D:maj, A:maj, C:maj, N, N, G:maj, D...","[C:maj, C:maj, C:maj, A:min, C:maj, F:maj, C:m..."
5,74,74,108,5cada0129fc5af34a460af2d,a-ha,"[Bb:min, Eb:min, Ab:maj, Bb:min, Bb:min, Eb:mi...",0,7,https://tabs.ultimate-guitar.com/tab/a-ha/youl...,Youll Never Get Over Me,...,Youll Never Get Over Me a-ha,"[{'st': 0, 'et': 0.395, 'ochord': 'F#:min'}, {...","[Gb:maj, Ab:maj, Bb:maj, Db:maj, Gb:maj, Ab:ma...","[E:min, A:min, D:maj, E:min, E:min, A:min, G:m...","[F:maj, G:maj, A:maj, C:maj, F:maj, G:maj, G:m...",4.5,7,78.2,"[Bb:min, N, N, N, Eb:min, Ab:maj, N, N, N, N, ...","[Gb:maj, Ab:maj, Bb:maj, Db:maj, Gb:maj, Ab:ma..."
6,75,75,109,5cada0129fc5af34a460af2f,ABBA,"[C:maj, G:maj, F:maj, C:maj, F:maj, C:maj, G:m...",0,91,https://tabs.ultimate-guitar.com/tab/abba/anda...,Andante Andante,...,Andante Andante ABBA,"[{'st': 0, 'et': 1.974, 'ochord': 'D'}, {'st':...","[D:maj, F:maj, C:maj, F:maj, Bb:maj, F:maj, C:...","[F:maj, C:maj, Bb:maj, F:maj, Bb:maj, F:maj, C...","[A:maj, C:maj, G:maj, C:maj, F:maj, C:maj, G:m...",5.0,108,42.6,"[C:maj, G:maj, F:maj, C:maj, F:maj, N, N, C:ma...","[D:maj, N, F:maj, C:maj, F:maj, Bb:maj, F:maj,..."
9,78,78,112,5cada0129fc5af34a460af37,ABBA,"[Gb:maj, B:maj, Gb:maj, B:maj, Gb:maj, B:maj, ...",0,3,https://tabs.ultimate-guitar.com/tab/abba/as_g...,As Good As New,...,As Good As New ABBA,"[{'st': 0, 'et': 0.65, 'ochord': 'B'}, {'st': ...","[B:maj, E:maj, B:maj, E:maj, B:maj, E:maj, A:m...","[B:maj, E:maj, B:maj, E:maj, B:maj, E:maj, A:m...","[C:maj, F:maj, C:maj, F:maj, C:maj, F:maj, N, ...",4.5,3,65.2,"[Gb:maj, B:maj, Gb:maj, B:maj, Gb:maj, B:maj, ...","[B:maj, N, E:maj, B:maj, E:maj, B:maj, E:maj, ..."
10,79,79,113,5cada0129fc5af34a460af39,ABBA,"[F:maj, Bb:maj, C:maj, F:maj, Bb:maj, C:maj, C...",0,4,https://tabs.ultimate-guitar.com/tab/abba/bang...,Bang A Boomerang,...,Bang A Boomerang ABBA,"[{'st': 0, 'et': 3.228, 'ochord': 'N'}, {'st':...","[N, F:maj, Bb:maj, C:maj, F:maj, C:maj, Bb:maj...","[F:maj, Bb:maj, C:maj, F:maj, Bb:maj, C:maj, C...","[N, C:maj, F:maj, G:maj, C:maj, G:maj, F:maj, ...",4.5,4,26.9,"[F:maj, N, Bb:maj, C:maj, F:maj, Bb:maj, C:maj...","[N, F:maj, Bb:maj, C:maj, F:maj, N, C:maj, N, ..."
11,80,80,114,5cada0129fc5af34a460af3d,ABBA,"[G:maj, G:maj, C:maj, G:maj, D:maj, F:maj, C:m...",0,4,https://tabs.ultimate-guitar.com/tab/abba/cass...,Cassandra

In [15]:
print(len(df2),'個Question(Song)')

8986 個Question(Song)


In [16]:
print(status(df2['chord_alignment_by_dtw'].str.len()),status(df2['songchord_alignment_by_dtw'].str.len()))



總數         8986.000000
最小值          28.000000
最小值位置     12081.000000
5%分位數        59.000000
25%分位數       85.000000
中位數         105.000000
75%分位數      126.000000
95%分位數      154.000000
均值          105.682840
最大值         210.000000
最大值位數      6699.000000
平均絕對偏差       23.222351
方差          809.974190
標準差          28.460045
偏度            0.052648
峰度           -0.491974
dtype: float64 總數         8986.000000
最小值          28.000000
最小值位置     12081.000000
5%分位數        59.000000
25%分位數       85.000000
中位數         105.000000
75%分位數      126.000000
95%分位數      154.000000
均值          105.682840
最大值         210.000000
最大值位數      6699.000000
平均絕對偏差       23.222351
方差          809.974190
標準差          28.460045
偏度            0.052648
峰度           -0.491974
dtype: float64


In [19]:
df2=df2.groupby('song_name').first()

# QA_train_df:QA模型訓練用的資料表 <br>


In [20]:
#QA_train_df, QA_test_df = train_test_split(df2, test_size=0.3)
QA_train_df=df2.copy()

In [21]:
print('QA_train_df長度:',len(QA_train_df))


QA_train_df長度: 8933


In [22]:
QA_train_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,artist,chord,error,rating,tab_href,tab_title,version,song_detector_chord,song_chord,chord_before_tf,song_chord_toCkey,star,visitors_0530,tab_dtw_cost,chord_alignment_by_dtw,songchord_alignment_by_dtw
song_name,,,,,,,,,,,,,,,,,,,,
0 Interest Jason Mraz,8420,8420,9127,5cada55a9fc5af34a4611660,Jason Mraz,"[N, C:maj, D:maj, Bb:maj, N, C:maj, D:maj, Bb:...",0,10,https://tabs.ultimate-guitar.com/tab/jason_mra...,0 Interest,Ver 1,"[{'st': 0, 'et': 0.627, 'ochord': 'N'}, {'st':...","[N, D:maj, F:maj, C:maj, A:min, G:maj, D:maj, ...","[F:maj, C:maj, D:maj, Bb:maj, F:maj, C:maj, D:...","[N, G:maj, N, F:maj, D:min, C:maj, G:maj, N, F...",4.5,11,47.6,"[N, N, C:maj, N, N, N, N, N, N, N, N, N, N, N,...","[N, D:maj, F:maj, C:maj, A:min, G:maj, D:maj, ..."
1 2 3 Sofia Reyes feat. Jason Derulo & De La Ghetto,8355,8355,9055,5cada54e9fc5af34a461158a,Sofia Reyes feat. Jason Derulo & De La Ghetto,"[F:min, Bb:min, F:min, Bb:min, F:min, Bb:min, ...",0,6,https://tabs.ultimate-guitar.com/tab/sofia_rey...,1 2 3,Ver 1,"[{'st': 0, 'et': 3.274, 'ochord': 'G#:min'}, {...","[Ab:maj, Db:maj, Ab:maj, Db:maj, Ab:maj, Db:ma...","[Ab:min, Db:min, Ab:min, Db:min, Ab:min, Db:mi...","[C:maj, F:maj, C:maj, F:maj, C:maj, F:maj, C:m...",4.5,6,56.2,"[F:min, Bb:min, F:min, Bb:min, F:min, Bb:min, ...","[Ab:maj, Db:maj, N, N, N, N, N, N, N, N, N, N,..."
10 Bon Iver,1981,1981,2168,5cada1409fc5af34a460c7ba,Bon Iver,"[Bb:min, Bb:min, Ab:maj, Eb:min, Bb:min, Ab:ma...",0,9,https://tabs.ultimate-guitar.com/tab/bon_iver/...,10,Ver 1,"[{'st': 0, 'et': 2.485, 'ochord': 'N'}, {'st':...","[N, N, N, Ab:maj, N, C:maj, Gb:maj, N, N, N, A...","[Ab:min, Ab:min, Gb:maj, Db:min, Ab:min, Gb:ma...","[N, N, N, G:maj, N, B:maj, F:maj, N, N, N, G:m...",4.5,10,38.0,"[Bb:min, N, Bb:min, Ab:maj, N, N, Eb:min, N, N...","[N, N, N, Ab:maj, N, C:maj, Gb:maj, N, N, N, A..."
10 Sorry Jonas Brothers,8828,8828,9576,5cada59e9fc5af34a4611b25,Jonas Brothers,"[D:maj, Bb:maj, C:maj, A:maj, Bb:maj, F:maj, C...",0,6,https://tabs.ultimate-guitar.com/tab/jonas_bro...,10 Sorry,Ver 1,"[{'st': 0, 'et': 0.929, 'ochord': 'N'}, {'st':...","[N, D:maj, Bb:maj, C:maj, C:maj, G:maj, Bb:maj...","[D:maj, Bb:maj, C:maj, A:maj, Bb:maj, F:maj, C...","[N, A:maj, F:maj, G:maj, G:maj, D:maj, F:maj, ...",4.0,6,29.8,"[D:maj, N, Bb:maj, C:maj, N, A:maj, Bb:maj, F:...","[N, D:maj, Bb:maj, C:maj, C:maj, G:maj, Bb:maj..."
100 Years Florence + The Machine,6491,6491,7050,5cada4149fc5af34a460ff1d,Florence + The Machine,"[Bb:maj, D:maj, N, G:maj, D:maj, D:maj, N, G:m...",0,6,https://tabs.ultimate-guitar.com/tab/florence_...,100 Years,Ver 2,"[{'st': 0, 'et': 2.926, 'ochord': 'N'}, {'st':...","[N, C:maj, B:maj, D:maj, Gb:maj, E:maj, D:maj,...","[Bb:maj, D:maj, F:maj, G:maj, D:maj, D:maj, F:...","[N, F:maj, E:maj, G:maj, B:maj, A:maj, G:maj, ...",5.0,7,40.4,"[Bb:maj, N, D:maj, N, N, G:maj, D:maj, D:maj, ...","[N, C:maj, B:maj, D:maj, Gb:maj, E:maj, D:maj,..."
1000 Things Jason Mraz,8421,8421,9128,5cada55a9fc5af34a4611666,Jason Mraz,"[C:maj, E:maj, C:maj, C:maj, D:maj, D:maj, C:m...",0,9,https://tabs.ultimate-guitar.com/tab/jason_mra...,1000 Things,Ver 2,"[{'st': 0, 'et': 0.488, 'ochord': 'N'}, {'st':...","[N, G:maj, B:maj, E:min, C:maj, G:maj, B:maj, ...","[G:maj, B:maj, G:maj, G:maj, A:maj, A:min, G:m...","[N, C:maj, E:maj, A:min, F:maj, C:maj, E:maj, ...",4.5,9,33.4,"[C:maj, N, E:maj, C:maj, C:maj, D:maj, D:maj, ...","[N, G:maj, B:maj, E:min, C:maj, G:maj, B:maj, ..."
1000 Times Sara Bareilles,15304,15304,16601,5cadab2f9fc5af34a4616ea1,Sara Bareilles,"[D:maj, A:maj, E:min, D:maj, A:maj, E:min, E:m...",0,106,https://tabs.ultimate-guitar.com/tab/sara_bare...,1000 Times,Ver 1,"[{'st': 0, 'et': 3.529, 'ochord': 'D'}, {'st':...","[D:maj, A:maj, E:maj, A:min, D:maj, A:maj, E:m...","[D:maj, A:maj, E:min, D:maj, A:maj, E:min, E:m...","[C:maj, G:maj, D:maj, G:min, C:maj, G:maj, D:m...",5.0,114,22.3,"[D:maj, A:maj, E:min, N, D:maj, A:maj, E:min, ...","[D:maj, A:maj, E:maj, A:min, D:maj, A:maj, E:m..."
10000 P

從QA_train_df製作三個資料: <br>
1.questions <br>
2.good_anwsers <br>
3.all_answers

In [30]:
chord_vector = {
               
        'C:maj':  [0,4,7],
        'Db:maj': [1,5,8],
        'Db:min': [1,4,8],
        'D:maj':  [2,7,9],
        'Eb:maj': [3,7,10],
        'Eb:min': [3,6,10],
        'E:maj':  [4,8,11],
        'E:min':  [4,7,11],
        'F:maj':  [0,5,9],
        'F:min':  [0,4,9],
        'Gb:maj': [1,6,10],
        'Gb:min': [1,6,9],
        'G:maj':  [2,7,11],
        'G:min':  [2,7,10],
        'Ab:maj': [0,3,8],
        'Ab:min': [3,8,11],
        'A:maj':  [1,4,9],
        'A:min':  [0,4,9],
        'Bb:maj': [2,5,10],
        'Bb:min': [1,5,10],
        'B:maj':  [2,6,11],
        'B:min':  [1,6,11]    
   
    }
chord_index={
    'C:maj':  1,
    'Db:maj': 2,
    'Db:min': 3,
    'D:maj':  4,
    'Eb:maj': 5,
    'Eb:min': 6,
    'E:maj':  7,
    'E:min':  8,
    'F:maj':  9,
    'F:min':  10,
    'Gb:maj': 11,
    'Gb:min': 12,
    'G:maj':  13,
    'G:min':  14,
    'Ab:maj': 15,
    'Ab:min': 16,
    'A:maj':  17,
    'A:min':  18,
    'Bb:maj': 19,
    'Bb:min': 20,
    'B:maj':  21,
    'B:min':  22    

}

## 0607:
Question是歌曲和弦轉成C調
GoodAnswer:Good吉他譜和弦不轉調

## 0609:
Question:原曲做Chord Detector拿到的和弦序列  
GoodAnswer:Good吉他譜和弦轉成原曲的調  
BadAnswer:訂星等與visitors的threshold  
想法:因為原曲是由Chord Detector得到的，與Good Tab的和弦序列有alignment的問題  
所以想要由DTW Cost較低的譜的和弦序列做為原曲和弦序列(Q)  
之後在Test data中選擇DTW Cost最低的吉他譜做為原曲和弦序列(Q)  
之後在未知資料中只要一支Chord Detector估出來的和弦序列去找DTW Cost最小的譜做為Good Tab

##061401:
    Question:原曲做Chord Detector拿到的和弦序列  
    GoodAnswer:Good吉他譜和弦轉成原曲的調  
    BadAnswer:訂星等與visitors的threshold 拿星等>=4.5的譜增加訓練強度

In [31]:
num=0
questions=[]
for index, row in QA_train_df.iterrows():
    num +=1
    if (num%1000)==0:
        print(num)
        
    
    questions_row=np.array([])
    for chord in row['song_chord']:
        
        index=chord_index.get(chord)
        if index is None:index=np.zeros(1)
        if index is not None:index=np.array(index)
        questions_row=np.hstack([questions_row, index]) if questions_row.size else index
    questions.append(questions_row)
    
num=0
good_answers=[]
    
for index, row in QA_train_df.iterrows():
    num +=1
    if (num%1000)==0:
        print(num)


    answers_row=np.array([])
    #0602:改成row['chord_before_transfer']，原row['chord']
    row_chord=row['chord']
   
#     if row['song_name'] in has_2ver[1000:]:
#         df_tmp=df[df.song_name==row['song_name']]
#         df_tmp=df_tmp[~(df_tmp.version==row['version'])]
#         min_index=df_tmp.rating.argmin()
#         row_chord=df_tmp.loc[min_index].chord
    
    for chord in row_chord:
        
        
        
            
        index=chord_index.get(chord)
        if index is None:index=np.zeros(1)
        if index is not None:index=np.array(index)
        answers_row=np.hstack([answers_row, index]) if answers_row.size else index
    good_answers.append(answers_row)



print('ok')

    

1000
2000
3000
4000
5000
6000
7000
8000
1000
2000
3000
4000
5000
6000
7000
8000
ok


從df裡拿到所有的吉他譜和弦當成All_answers <br>

之後要從All_answers裡抽樣bad answers

0602:All_answers拿轉調前的吉他和弦欄位:chord_bf_transfer，看會不會因為噪音較多能抽出bad answers <br>
0609:BadAnswer:訂星等與visitors的threshold  
061401:BadAnswer:訂星等與visitors的threshold

In [35]:
all_answers=[]
num=0
#Threshold
df_filtered=df[(df['star']>=4.5)  ]

for row in df_filtered.chord:
    num +=1
    if (num%1000)==0:
        print(num)


    answers_row=np.array([])
    for chord in row:

        index=chord_index.get(chord)
        if index is None:index=np.zeros(1)
        if index is not None:index=np.array(index)
        answers_row=np.hstack([answers_row, index]) if answers_row.size else index
    all_answers.append(answers_row)
print('ok')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
ok


In [36]:
def find_max_length_of_songs(Q,A):
    max_length=0
    for i in Q:
        if max_length<len(i):max_length=len(i)
    for i in A:
        if max_length<len(i):max_length=len(i)
    return max_length

要知道最多和弦的歌曲，和弦數量是多少，然後把每首歌的和弦數目pad成一樣長

In [37]:

find_max_length_of_songs(questions,good_answers)

158

In [38]:
#pad成158個row(2d matrix,149X3)
def pad_QA(QA):
    data=QA[:]
    
    for i in range(len(data)):
        if (i%1000)==0:print(i)
        
        a=data[i]
        padding_length=158-len(a)
      
        data[i]=np.pad(a, [(0, padding_length)], mode='constant')
      
           
    return data

In [39]:
questions=pad_QA(questions)
good_answers=pad_QA(good_answers)
print('ok')

0
1000
2000
3000
4000
5000
6000
7000
8000
0
1000
2000
3000
4000
5000
6000
7000
8000
ok


In [40]:
all_answers=pad_QA(all_answers)
print('ok')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
ok


In [42]:
#檢查padding後的資料長度
len(all_answers[14000])

158

In [45]:
import pickle

out = open("../../QA_data/061401/questions","wb")
pickle.dump(questions, out)
out = open("../../QA_data/061401/good_answers","wb")
pickle.dump(good_answers, out)
del out
gc.collect()
print('ok')

ok


In [47]:
import pickle

out = open("../../QA_data/061401/all_answers","wb")
pickle.dump(all_answers, out)
print('ok')

ok
